In [ ]:
#| default_exp distributed.fugue

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# FugueBackend

> The computational efficiency of `StatsForecast` can be tracked to its two core components:<br>1. Its `models` written in NumBa that optimizes Python code to reach C speeds.<br>2. Its `core.StatsForecast` class that enables distributed computing.<br><br>Here we use [Fugue](https://github.com/fugue-project/fugue) which is a unified interface for `Dask` and `Spark`.<br><br>

In [ ]:
#| hide
from fastcore.test import test_eq
from nbdev.showdoc import add_docs, show_doc

In [ ]:
#| export
from typing import Any, Dict, List

import numpy as np
import pandas as pd
try:
    from fugue import transform
except ModuleNotFoundError as e:
    msg = (
        f'{e}. To use fugue you have to install it.'
        'Please run `pip install fugue`. '
    )
    raise ModuleNotFoundError(msg) from e
from statsforecast.core import StatsForecast
from statsforecast.distributed.core import ParallelBackend
from triad import Schema

In [ ]:
#| export
class FugueBackend(ParallelBackend):
    """FugueBackend for Distributed Computation.
    [Source code](https://github.com/Nixtla/statsforecast/blob/main/statsforecast/distributed/fugue.py).

    This class uses [Fugue](https://github.com/fugue-project/fugue) backend capable of distributing 
    computation on Spark and Dask without any rewrites.

    **Parameters:**<br>
    `engine`: fugue.ExecutionEngine, a selection between spark and dask.<br>
    `conf`: fugue.Config, engine configuration.<br>
    `**transform_kwargs`: additional kwargs for Fugue's transform method.<br>

    **Notes:**<br>
    A short introduction to Fugue, with examples on how to scale pandas code to scale pandas 
    based code to Spark or Dask is available [here](https://fugue-tutorials.readthedocs.io/tutorials/quick_look/ten_minutes.html).
    """
    def __init__(
            self, 
            engine: Any = None,
            conf: Any = None,
            **transform_kwargs: Any
        ):        
        self._engine = engine
        self._conf = conf
        self._transform_kwargs = dict(transform_kwargs)

    def __getstate__(self) -> Dict[str, Any]:
        return {}

    def forecast(
            self, 
            df,
            models,
            freq,
            **kwargs: Any,
        ) -> Any:
        """Memory Efficient core.StatsForecast predictions with FugueBackend.

        This method uses Fugue's transform function, in combination with 
        `core.StatsForecast`'s forecast to efficiently fit a list of StatsForecast models.

        **Parameters:**<br>
        `df`: pandas.DataFrame, with columns [`unique_id`, `ds`, `y`] and exogenous.<br>
        `freq`: str, frequency of the data, [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).<br>
        `models`: List[typing.Any], list of instantiated objects `StatsForecast.models`.<br>
        `**kwargs`: Additional `core.StatsForecast` parameters. Example forecast horizon `h`.<br>

        **Returns:**<br>
        `fcsts_df`: pandas.DataFrame, with `models` columns for point predictions and probabilistic
        predictions for all fitted `models`.<br>
        
        **References:**<br>
        For more information check the 
        [Fugue's transform](https://fugue-tutorials.readthedocs.io/tutorials/beginner/introduction.html#fugue-transform)
        tutorial.<br>
        The [core.StatsForecast's forecast](https://nixtla.github.io/statsforecast/core.html#statsforecast.forecast)
        method documentation.<br>
        Or the list of available [StatsForecast's models](https://nixtla.github.io/statsforecast/models.html).
        """
        schema = "*-y+" + str(self._get_output_schema(models))
        return transform(
            df,
            self._forecast_series,
            params=dict(models=models, freq=freq, kwargs=kwargs),
            schema=schema,
            partition={"by": "unique_id"},
            engine=self._engine,
            engine_conf=self._conf,
            **self._transform_kwargs,
        )

    def cross_validation(
            self, 
            df,
            models,
            freq,
            **kwargs: Any, 
        ) -> Any:
        """Temporal Cross-Validation with core.StatsForecast and FugueBackend.

        This method uses Fugue's transform function, in combination with 
        `core.StatsForecast`'s cross-validation to efficiently fit a list of StatsForecast 
        models through multiple training windows, in either chained or rolled manner.

        `StatsForecast.models`' speed along with Fugue's distributed computation allow to 
        overcome this evaluation technique high computational costs. Temporal cross-validation 
        provides better model's generalization measurements by increasing the test's length 
        and diversity.

        **Parameters:**<br>
        `df`: pandas.DataFrame, with columns [`unique_id`, `ds`, `y`] and exogenous.<br>
        `freq`: str, frequency of the data, [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).<br>
        `models`: List[typing.Any], list of instantiated objects `StatsForecast.models`.<br>

        **Returns:**<br>
        `fcsts_df`: pandas.DataFrame, with `models` columns for point predictions and probabilistic
        predictions for all fitted `models`.<br>
        
        **References:**<br>
        The [core.StatsForecast's cross validation](https://nixtla.github.io/statsforecast/core.html#statsforecast.cross_validation)
        method documentation.<br>
        [Rob J. Hyndman and George Athanasopoulos (2018). "Forecasting principles and practice, Temporal Cross-Validation"](https://otexts.com/fpp3/tscv.html).
        """
        schema = "*-y+" + str(self._get_output_schema(models, mode="cv"))
        return transform(
            df,
            self._cv,
            params=dict(models=models, freq=freq, kwargs=kwargs),
            schema=schema,
            partition={"by": "unique_id"},
            engine=self._engine,
            engine_conf=self._conf,
            **self._transform_kwargs,
        )

    def _forecast_series(self, df: pd.DataFrame, models, freq, kwargs) -> pd.DataFrame:
        tdf = df.set_index("unique_id")
        model = StatsForecast(df=tdf, models=models, freq=freq, n_jobs=1)
        return model.forecast(**kwargs).reset_index()

    def _cv(self, df: pd.DataFrame, models, freq, kwargs) -> pd.DataFrame:
        tdf = df.set_index("unique_id")
        model = StatsForecast(df=tdf, models=models, freq=freq, n_jobs=1)
        return model.cross_validation(**kwargs).reset_index()

    def _get_output_schema(self, models, mode="forecast") -> Schema:
        cols: List[Any]
        cols = [(repr(model), np.float32) for model in models]
        if mode == "cv":
            cols = [("cutoff", "datetime"), ("y", np.float32)] + cols
        return Schema(cols)

In [ ]:
show_doc(FugueBackend, title_level=3)

---

### FugueBackend

>      FugueBackend (engine:Any=None, conf:Any=None, **transform_kwargs:Any)

FugueBackend for Distributed Computation.
[Source code](https://github.com/Nixtla/statsforecast/blob/main/statsforecast/distributed/fugue.py).

This class uses [Fugue](https://github.com/fugue-project/fugue) backend capable of distributing 
computation on Spark and Dask without any rewrites.

**Parameters:**<br>
`engine`: fugue.ExecutionEngine, a selection between spark and dask.<br>
`conf`: fugue.Config, engine configuration.<br>
`**transform_kwargs`: additional kwargs for Fugue's transform method.<br>

**Notes:**<br>
A short introduction to Fugue, with examples on how to scale pandas code to scale pandas 
based code to Spark or Dask is available [here](https://fugue-tutorials.readthedocs.io/tutorials/quick_look/ten_minutes.html).

In [ ]:
show_doc(FugueBackend.forecast, title_level=3)

---

### FugueBackend.forecast

>      FugueBackend.forecast (df, models, freq, **kwargs:Any)

Memory Efficient core.StatsForecast predictions with FugueBackend.

This method uses Fugue's transform function, in combination with 
`core.StatsForecast`'s forecast to efficiently fit a list of StatsForecast models.

**Parameters:**<br>
`df`: pandas.DataFrame, with columns [`unique_id`, `ds`, `y`] and exogenous.<br>
`freq`: str, frequency of the data, [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).<br>
`models`: List[typing.Any], list of instantiated objects `StatsForecast.models`.<br>
`**kwargs`: Additional `core.StatsForecast` parameters. Example forecast horizon `h`.<br>

**Returns:**<br>
`fcsts_df`: pandas.DataFrame, with `models` columns for point predictions and probabilistic
predictions for all fitted `models`.<br>

**References:**<br>
For more information check the 
[Fugue's transform](https://fugue-tutorials.readthedocs.io/tutorials/beginner/introduction.html#fugue-transform)
tutorial.<br>
The [core.StatsForecast's forecast](https://nixtla.github.io/statsforecast/core.html#statsforecast.forecast)
method documentation.<br>
Or the list of available [StatsForecast's models](https://nixtla.github.io/statsforecast/models.html).

In [ ]:
show_doc(FugueBackend.cross_validation, title_level=3)

---

### FugueBackend.cross_validation

>      FugueBackend.cross_validation (df, models, freq, **kwargs:Any)

Temporal Cross-Validation with core.StatsForecast and FugueBackend.

This method uses Fugue's transform function, in combination with 
`core.StatsForecast`'s cross-validation to efficiently fit a list of StatsForecast 
models through multiple training windows, in either chained or rolled manner.

`StatsForecast.models`' speed along with Fugue's distributed computation allow to 
overcome this evaluation technique high computational costs. Temporal cross-validation 
provides better model's generalization measurements by increasing the test's length 
and diversity.

**Parameters:**<br>
`df`: pandas.DataFrame, with columns [`unique_id`, `ds`, `y`] and exogenous.<br>
`freq`: str, frequency of the data, [panda's available frequencies](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases).<br>
`models`: List[typing.Any], list of instantiated objects `StatsForecast.models`.<br>

**Returns:**<br>
`fcsts_df`: pandas.DataFrame, with `models` columns for point predictions and probabilistic
predictions for all fitted `models`.<br>

**References:**<br>
The [core.StatsForecast's cross validation](https://nixtla.github.io/statsforecast/core.html#statsforecast.cross_validation)
method documentation.<br>
[Rob J. Hyndman and George Athanasopoulos (2018). "forecasting principles and practice, Temporal Cross-Validation"](https://otexts.com/fpp3/tscv.html).

## Dask Distributed Predictions

Here we provide an example for the distribution of the `StatsForecast` predictions using `Fugue` to execute the code in a Dask cluster.

To do it we instantiate the `FugueBackend` class with a `DaskExecutionEngine`.

In [ ]:
#| eval: false
from dask.distributed import Client
from fugue_dask import DaskExecutionEngine
from statsforecast.models import Naive
from statsforecast.utils import generate_series

# Generate Synthetic Panel Data
df = generate_series(10).reset_index()
df['unique_id'] = df['unique_id'].astype(str)

# Instantiate FugueBackend with DaskExecutionEngine
dask_client = Client()
engine = DaskExecutionEngine(dask_client=dask_client)
fcst = FugueBackend(engine=engine, as_local=True)

### Distributed Forecast

For extremely fast distributed predictions we use FugueBackend.forecast method that operates like the original [StatsForecast.forecast](https://nixtla.github.io/statsforecast/core.html#statsforecast.forecast) method.

It receives as input a pandas.DataFrame with columns [`unique_id`,`ds`,`y`] and exogenous, where the `ds` (datestamp) column should be of a format expected by Pandas. The `y` column must be numeric, and represents the measurement we wish to forecast. And the `unique_id` uniquely identifies the series in the panel data.

In [ ]:
#| eval: false

# Distributed predictions with FugueBackend.
fcst.forecast(df=df, models=[Naive()], freq='D', h=12)

### Distributed Cross-Validation

For extremely fast distributed temporcal cross-validation we use `FugueBackend.forecast` method that operates like the original [StatsForecast.cross_validation](https://nixtla.github.io/statsforecast/core.html#statsforecast) method.

In [ ]:
#| eval: false

# Distributed cross-validation with FugueBackend.
fcst.cross_validation(df=df, models=[Naive()], freq='D', h=12, n_windows=2)

In [ ]:
#| hide
#| eval: false
from statsforecast.models import Naive
from statsforecast.utils import generate_series

# Generate Synthetic Panel Data.
df = generate_series(10).reset_index()
df['unique_id'] = df['unique_id'].astype(str)

# Instantiate FugueBackend and distribute predictions.
backend = FugueBackend()
fcst_fugue = backend.forecast(df=df, models=[Naive()], freq='D', h=12)
fcst_stats = StatsForecast(models=[Naive()], freq='D').forecast(df=df, h=12)
test_eq(fcst_fugue, fcst_stats.reset_index())

# Distribute cross-validation predictions.
fcst_fugue = backend.cross_validation(df=df, models=[Naive()], freq='D', h=12)
fcst_stats = StatsForecast(models=[Naive()], freq='D').cross_validation(df=df, h=12)
test_eq(fcst_fugue, fcst_stats.reset_index())